Código - Simulação da Cinética Enzimática
===========================================================

**Autores:**  

João Gabriel Lima Marengo  
Joaquim Junior Ferola Fonseca  
Sophia Alves da Silva

---

## 1. Importação das Bibliotecas

Primeiramente, importamos as bibliotecas que serão usadas para cálculos, gráficos e construção da interface.

> ⚠️ Caso ainda não tenha as bibliotecas instaladas, execute o comando abaixo:

In [ ]:
!pip install plotly dash numpy

Após garantir que as bibliotecas estão instaladas, podemos iniciar a importação:

In [8]:
import plotly.graph_objects as go
from dash import Dash, dcc, html, Input, Output
import numpy as np
import math


## 2. Codificação da Função Base

Após importar as bibliotecas necessárias, o próximo passo é definir a função `velocidade_da_reacao`, responsável por calcular a taxa da reação enzimática com base em múltiplos parâmetros físico-químicos.

Essa função será usada para calcular a velocidade da reação enzimática no código que se segue, permitindo analisar como diferentes parâmetros (temperatura, pH, concentração, etc.) influenciam o comportamento da enzima.

In [9]:
def velocidade_da_reacao(S, T, pH, pKa, pKb, Vmax_ref, Ea, T_ref, Km_ref, deltaH, T_opt, sigma):
    T_K = T + 273.15
    T_ref_K = T_ref + 273.15
    T_opt_K = T_opt + 273.15

    term1 = 1 / (1 + 10**(pKa - pH) + 10**(pH - pKb))
    exp1 = math.exp((-Ea * 1000 / 8.314) * (1 / T_K - 1 / T_ref_K))
    exp2 = math.exp((deltaH * 1000 / 8.314) * (1 / T_K - 1 / T_ref_K))
    fator_f = math.exp(-((T_K - T_opt_K) / sigma)**2)

    numerador = Vmax_ref * exp1 * fator_f * S
    denominador = Km_ref * exp2 + S

    return term1 * (numerador / denominador)


## 3. Construção da Interface

Após definirmos a função que calcula a velocidade da reação, construímos a interface da aplicação com a biblioteca **Dash**, dividindo-a em duas colunas: à esquerda, os campos de entrada para ajuste dos parâmetros da reação; à direita, um gráfico 3D interativo gerado com Plotly que mostra os efeitos desses parâmetros.

A linha `app = Dash(__name__)` inicializa o app, conectando a interface visual à lógica de cálculo do backend. O argumento `__name__` indica o local do script para garantir o funcionamento correto.

Para um visual claro e funcional, aplicamos estilos baseados no *flat design*, definidos em dicionários Python para facilitar sua aplicação:

* **label\_style:** rótulos em negrito, cor escura e espaçamento para melhor leitura;
* **input\_style:** campos de entrada com largura total, bordas arredondadas e transição suave na borda;
* **container\_style:** contêiner principal com largura limitada, centralizado, fundo suave e sombra leve;
* **title\_style:** título centralizado, destacado e espaçado.

Usamos o componente html.Div para estruturar e agrupar esses elementos, controlando o posicionamento dos conteúdos e organizando a interface com os controles de entrada à esquerda e o gráfico interativo à direita.

A seguir, o código que implementa essa estrutura e estilo da interface:

In [10]:
app = Dash(__name__)

label_style = {
    'color': '#333333',
    'fontWeight': 'bold',
    'fontSize': '14px',
    'marginBottom': '6px',
    'display': 'block'
}

input_style = {
    'width': '100%',
    'padding': '8px 12px',
    'border': '1px solid #ccc',
    'borderRadius': '6px',
    'fontSize': '14px',
    'fontWeight': '400',
    'outline': 'none',
    'boxShadow': 'none',
    'transition': 'border-color 0.3s ease'
}

container_style = {
    'maxWidth': '1100px',
    'margin': '40px auto',
    'padding': '30px',
    'backgroundColor': '#f5f5f7',
    'borderRadius': '12px',
    'fontFamily': 'Trebuchet MS, sans-serif',
    'color': '#1d1d1f',
    'boxShadow': '0 4px 12px rgba(0,0,0,0.05)'
}

title_style = {
    'textAlign': 'center',
    'color': '#1d1d1f',
    'marginBottom': '40px',
    'fontWeight': '600',
    'fontSize': '28px'
}

app.layout = html.Div([
    html.H1("Simulação - Cinética Enzimática", style=title_style),

    html.Div([
        html.Div([
            html.Div([
                html.Label("Concentração do Substrato (mol/L)", style=label_style),
                dcc.Input(id='input_s', type='number', value=10, step=0.1, style=input_style)
            ], style={'marginBottom': '10px'}),
            html.Div([
                html.Label("Constante de Dissociação do Grupo Ácido (pKa)", style=label_style),
                dcc.Input(id='input_pka', type='number', value=6.5, step=0.1, style=input_style)
            ], style={'marginBottom': '10px'}),
            html.Div([
                html.Label("Constante de Dissociação do Grupo Básico (pKb)", style=label_style),
                dcc.Input(id='input_pkb', type='number', value=8, step=0.1, style=input_style)
            ], style={'marginBottom': '10px'}),
            html.Div([
                html.Label("Velocidade Máxima da Reação (mol/L.s)", style=label_style),
                dcc.Input(id='input_vmax_ref', type='number', value=32, style=input_style)
            ], style={'marginBottom': '10px'}),
            html.Div([
                html.Label("Energia de Ativação da Reação (kJ/mol)", style=label_style),
                dcc.Input(id='input_ea', type='number', value=50, style=input_style)
            ], style={'marginBottom': '10px'}),
            html.Div([
                html.Label("Temperatura de Referência (°C)", style=label_style),
                dcc.Input(id='input_tref', type='number', value=25, style=input_style)
            ], style={'marginBottom': '10px'}),
            html.Div([
                html.Label("Constante de Michaelis-Menten (mol/L)", style=label_style),
                dcc.Input(id='input_km_ref', type='number', value=0.4, style=input_style)
            ], style={'marginBottom': '10px'}),
            html.Div([
                html.Label("Variação de Entalpia (kJ/mol)", style=label_style),
                dcc.Input(id='input_deltah', type='number', value=-20, style=input_style)
            ], style={'marginBottom': '10px'}),
            html.Div([
                html.Label("Temperatura Ótima em Condições Ideais (°C)", style=label_style),
                dcc.Input(id='input_topt', type='number', value=37, style=input_style)
            ], style={'marginBottom': '10px'}),
            html.Div([
                html.Label("Sensibilidade Térmica (°C)", style=label_style),
                dcc.Input(id='input_sigma', type='number', value=10, style=input_style)
            ], style={'marginBottom': '10px'}),
        ], style={
            'flex': '0 0 380px',
            'paddingRight': '30px',
            'overflowY': 'auto',
            'maxHeight': '750px',
            'boxSizing': 'border-box'
        }),

        html.Div([
            dcc.Graph(id='graph-3d', style={'height': '700px'})
        ], style={'flex': '1'})
    ], style={'display': 'flex', 'flexDirection': 'row'})
], style=container_style)

## 4. Callback para Atualização do Gráfico

Na sequência, definimos um callback, um dos recursos centrais da biblioteca Dash. Um callback é uma função que permite ligar componentes da interface gráfica aos dados e cálculos de backend. Em outras palavras, ele escuta mudanças nos componentes de entrada e atualiza automaticamente os elementos de saída, como gráficos ou tabelas.

No nosso projeto, o callback é ativado sempre que o usuário altera algum dos parâmetros da reação (como concentração, temperatura, pH etc.). Quando isso ocorre, a função `update_graph_3d` é chamada para recalcular e redesenhar o gráfico interativo.

Para entender melhor como essa função trabalha, apresentamos a seguir uma descrição detalhada das etapas realizadas pelo callback:

### 4.1. O que o callback faz?

#### 4.1.1. Gera grades de valores de temperatura e pH:

In [39]:
# T_vals = np.linspace(start, stop, num)
# pH_vals = np.linspace(start, stop, num)
# T_grid, pH_grid = np.meshgrid(T_vals, pH_vals)

* `np.linspace(start, stop, num)` cria uma sequência de `num` valores igualmente espaçados entre `start` e `stop`.

* `np.meshgrid` gera duas matrizes que representam a grade de combinações entre as temperaturas (`T_vals`) e os valores de pH (`pH_vals`).

#### 4.1.2. Calcula a velocidade da reação para cada par (T, pH) da malha:

In [ ]:
# v_grid = np.vectorize(lambda T, pH: velocidade_da_reacao(...))(T_grid, pH_grid)

* `np.vectorize` transforma a função `velocidade_da_reacao`, que normalmente opera em um par de valores, para aceitar essas matrizes e aplicar a função elemento a elemento em toda a grade (`T_grid`, `pH_grid`).

* O resultado é uma matriz `v_grid` com as velocidades calculadas para cada combinação de temperatura e pH.

#### 4.1.3. Cria um gráfico de superfície 3D interativo com os valores de velocidade calculados:

In [ ]:
# fig = go.Figure(data=[go.Surface(...)])

* `go.Surface` cria uma superfície tridimensional baseada nos dados de `z` (velocidade) em função dos eixos `x` (temperatura) e `y` (pH).

* `go.Figure` encapsula o gráfico, que pode ser manipulado e exibido no Dash.

#### 4.1.4. Aplica configurações estéticas e eixos ao gráfico:

In [ ]:
# fig.update_layout(...)

A função `fig.update_layout(...)` é usada para ajustar aspectos importantes da aparência do gráfico, incluindo:

* Títulos dos eixos para facilitar a compreensão dos dados
* Estilo e formato da fonte para melhorar a legibilidade
* Limites das faixas dos eixos para focar nos valores relevantes
* Cores de fundo e das grades para tornar o gráfico mais claro e visualmente agradável

Esse processo garante que qualquer mudança feita pelo usuário nos inputs seja imediatamente refletida no gráfico, tornando a visualização responsiva e exploratória.

O código completo do callback está logo abaixo:

In [11]:
@app.callback(
    Output('graph-3d', 'figure'),
    Input('input_s', 'value'),
    Input('input_pka', 'value'),
    Input('input_pkb', 'value'),
    Input('input_vmax_ref', 'value'),
    Input('input_ea', 'value'),
    Input('input_tref', 'value'),
    Input('input_km_ref', 'value'),
    Input('input_deltah', 'value'),
    Input('input_topt', 'value'),
    Input('input_sigma', 'value')
)
def update_graph_3d(S, pKa, pKb, Vmax_ref, Ea, T_ref, Km_ref, deltaH, T_opt, sigma):
    T_vals = np.linspace(10, 50, 40)
    pH_vals = np.linspace(4, 10, 40)
    T_grid, pH_grid = np.meshgrid(T_vals, pH_vals)
    v_grid = np.vectorize(lambda T, pH: velocidade_da_reacao(S, T, pH, pKa, pKb, Vmax_ref, Ea, T_ref, Km_ref, deltaH, T_opt, sigma))(T_grid, pH_grid)

    fig = go.Figure(data=[go.Surface(
        z=v_grid,
        x=T_vals,
        y=pH_vals,
        colorscale='Viridis',
        colorbar=dict(title='Velocidade (mol/L·s)')
    )])

    fig.update_layout(
        title="Velocidade da Reação Enzimática",
        title_x=0.5,
        title_font=dict(
            size=20,
            color='#1d1d1f',
            family='Arial, sans-serif',
            weight='bold'
        ),
        scene=dict(
            xaxis_title='Temperatura (°C)',
            yaxis_title='pH',
            zaxis_title='Velocidade (mol/L·s)',
            xaxis=dict(showgrid=True, gridcolor='#e1e1e1', zerolinecolor='#e1e1e1', linecolor='#ccc'),
            yaxis=dict(showgrid=True, gridcolor='#e1e1e1', zerolinecolor='#e1e1e1', linecolor='#ccc'),
            zaxis=dict(showgrid=True, gridcolor='#e1e1e1', zerolinecolor='#e1e1e1', linecolor='#ccc'),
            xaxis_range=[10, 50],
            yaxis_range=[4, 10],
        ),
        paper_bgcolor='white',
        plot_bgcolor='white',
        font=dict(color='#1d1d1f')
    )

    return fig

Esse callback torna possível a visualização dinâmica da cinética enzimática, reforçando o objetivo do projeto: explorar os efeitos combinados de diversos parâmetros sobre a velocidade da reação em um ambiente interativo usando Dash e Plotly.

## 5. Execução da Aplicação

Após definirmos todos os componentes da aplicação (função de cálculo, layout da interface e lógica de atualização), estamos prontos para executar o app Dash localmente.

Para isso, basta rodar a célula abaixo:

In [ ]:
if __name__ == '__main__':
    app.run(debug=True, port=8080)

> ⚠️ Caso ocorra um erro indicando que a porta `8080` já está em uso, basta substituir esse número por outro maior e livre no comando `app.run_server(port=8080)`. Isso garantirá que a aplicação rode sem conflitos.

Em alguns ambientes, a interface interativa pode ser exibida diretamente na célula, sem necessidade de abrir o navegador. Contudo, recomenda-se abrir o aplicativo no navegador pelo endereço indicado e utilizar a tecla **F11** para ativar o modo de tela cheia, garantindo uma melhor visualização e experiência de uso:

🔗 [http://127.0.0.1:8080](http://127.0.0.1:8080)

> ⚠️ Lembre-se de que, se você alterou o número da porta, também deve atualizar o final do endereço no navegador para corresponder ao novo número.

Ao acessar o link, você encontrará uma interface com dois painéis:

* À esquerda, estão os controles onde é possível ajustar os parâmetros da reação enzimática (como concentração, temperatura, pH, energia de ativação, etc.);

* À direita, será exibido um gráfico 3D interativo, que mostra como a velocidade da reação varia em função da temperatura e do pH.

A cada mudança nos inputs, o gráfico será automaticamente atualizado, graças ao callback definido anteriormente, permitindo explorar visualmente o comportamento da cinética enzimática em diferentes condições experimentais.